
# System Q&A z Fine-Tuningiem T5 i Retrieverem (FAISS)

**Cel zadania:**  
Stwórz system pytanie-odpowiedź (Q&A), który:
- Wykorzystuje **Twój własny zbiór danych** (CSV: pytania, konteksty, odpowiedzi),
- Trenuje model **T5-small** na Twoim zbiorze (fine-tuning),
- Wyszukuje kontekst za pomocą **retrievera FAISS**,
- Generuje odpowiedzi z pomocą fine-tunowanego modelu.

---

## Zadania (3 punkty):
1) Przygotowanie własnych danych (CSV)  
2) Fine-tuning modelu T5-small  
3) Stworzenie retrievera FAISS + generowanie odpowiedzi  
4) Testy i analiza wyników  
5) Raport PDF z wnioskami

Fine tuning = 1 punkt   
RAG = 1 punkt   
PDF = 1 punkt   

**Uwaga:** Fine-tuning powinien być wykonany na **innym temacie lub zakresie pytań** niż dokumenty w retrieverze FAISS.
Nie używaj tych samych danych w obu miejscach!


##  Przygotowanie danych

Przygotuj własny plik CSV z danymi: `question`, `context`, `answer`.

Przykładowy format (przygotuj samodzielnie!):

| question                      | context                                    | answer       |
|------------------------------|--------------------------------------------|--------------|
| What is the capital of France?| The capital of France is Paris.             | Paris        |
| What is the currency of USA? | The currency of the USA is the dollar.      | The dollar   |

In [1]:
!pip install langchain-community faiss-cpu
#!pip install transformers datasets sentence-transformers

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
import numpy as np

In [20]:
import random

In [3]:
# Wczytaj swój plik CSV
df = pd.read_csv("wynalazki_wynalazcy_dataset.csv")
df.head()

,question,context,answer
0,Who came up with the idea of the internet?,Tim Berners-Lee invented the World Wide Web in...,Tim Berners-Lee
1,Who created the battery?,Alessandro Volta invented the battery in 1800.,Alessandro Volta
2,Who invented the printing press?,Johannes Gutenberg invented the printing press...,Johannes Gutenberg
3,Who created the vaccination?,Edward Jenner developed the smallpox vaccine i...,Edward Jenner
4,Who came up with the idea of the microscope?,Anton van Leeuwenhoek improved the microscope ...,Anton van Leeuwenhoek



## Fine-tuning modelu T5-small
**Uwaga:** Fine-tuning powinien być wykonany na **innym temacie lub zakresie pytań** niż dokumenty w retrieverze FAISS.
Nie używaj tych samych danych w obu miejscach!

In [4]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
dataset = Dataset.from_pandas(df)

def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['context'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["answer"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
training_args = TrainingArguments(
    output_dir="./t5-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=1,
    report_to=["tensorboard"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=75, training_loss=5.64665283203125, metrics={'train_runtime': 910.6464, 'train_samples_per_second': 0.329, 'train_steps_per_second': 0.082, 'total_flos': 40602540441600.0, 'train_loss': 5.64665283203125, 'epoch': 3.0})

In [15]:
# Sprawdź czy model jest w stanie odpowiedzieć na pytania z Twojego zbioru danych!
# Możesz zmodyfikować parametry treningu
if 'df' in locals() and not df.empty:
    num_questions_to_test = min(5, len(df))

    print(f"\n--- Testowanie {num_questions_to_test} przykładowych danych z zestawu treningowego ---")

    # Iteruj przez wybraną liczbę pytań
    for i in range(num_questions_to_test):
        sample_question = df.iloc[i]['question']
        sample_context = df.iloc[i]['context']
        sample_answer = df.iloc[i]['answer']

        print(f"\n--- Test #{i+1} ---")
        print(f"Pytanie z zestawu: {sample_question}")
        print(f"Kontekst z zestawu: {sample_context}")
        print(f"Oryginalna odpowiedź z zestawu: {sample_answer}")

        # Przygotuj input dla modelu
        input_text = f"question: {sample_question} context: {sample_context}"
        input_ids = tokenizer.encode(input_text, return_tensors='pt')

        # Wygeneruj odpowiedź
        # Używamy modelu, który jest dostępny w zmiennej 'model'
        output = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)

        print(f"Wygenerowana odpowiedź modelu: {generated_answer}")
        print(f"--- Koniec Testu #{i+1} ---")

else:
    print("DataFrame 'df' jest niedostępny lub pusty. Nie można przeprowadzić testu na danych treningowych.")



--- Testowanie 5 przykładowych danych z zestawu treningowego ---

--- Test #1 ---
Pytanie z zestawu: Who came up with the idea of the internet?
Kontekst z zestawu: Tim Berners-Lee invented the World Wide Web in 1989.
Oryginalna odpowiedź z zestawu: Tim Berners-Lee
Wygenerowana odpowiedź modelu: 
--- Koniec Testu #1 ---

--- Test #2 ---
Pytanie z zestawu: Who created the battery?
Kontekst z zestawu: Alessandro Volta invented the battery in 1800.
Oryginalna odpowiedź z zestawu: Alessandro Volta
Wygenerowana odpowiedź modelu: 
--- Koniec Testu #2 ---

--- Test #3 ---
Pytanie z zestawu: Who invented the printing press?
Kontekst z zestawu: Johannes Gutenberg invented the printing press around 1440.
Oryginalna odpowiedź z zestawu: Johannes Gutenberg
Wygenerowana odpowiedź modelu: 
--- Koniec Testu #3 ---

--- Test #4 ---
Pytanie z zestawu: Who created the vaccination?
Kontekst z zestawu: Edward Jenner developed the smallpox vaccine in 1796.
Oryginalna odpowiedź z zestawu: Edward Jenner
Wyge


## Retriever (FAISS) + Generowanie odpowiedzi

Zbuduj system, który wyszukuje kontekst w dokumentach i generuje odpowiedzi.

Przygotuj własną listę dokumentów do wyszukiwania (minimum 20 zdań).

**Uwaga:** Twoja baza wiedzy (lista dokumentów) w retrieverze **nie powinna być taka sama jak dane do fine-tuningu**.
Celem jest sprawdzenie, jak system radzi sobie z nowymi pytaniami i nową bazą wiedzy.

In [16]:
# Określ nazwę pliku CSV
csv_file_path = "sentences.csv"  # Zmień, jeśli nazwa pliku jest inna

# Wczytaj plik CSV, zakładając brak nagłówka i jedną kolumnę
try:
    # header=None - brak nagłówka
    # names=['sentence'] - nadajemy nazwę dla kolumny (może być dowolna)
    # squeeze=True - wczytaj jako Series, jeśli jest tylko jedna kolumna (w nowszych pandas deprecated, lepiej wybrać kolumnę po nazwie)
    df_sentences = pd.read_csv(csv_file_path, header=None, names=['sentence'])
    print(f"Pomyślnie wczytano plik: {csv_file_path}")

    # Sprawdź, czy DataFrame nie jest pusty po wczytaniu
    if not df_sentences.empty:
        # Pobierz zdania z kolumny i przekonwertuj na listę
        documents = df_sentences['sentence'].tolist()
        print(f"Wczytano {len(documents)} dokumentów.")
        # Możesz wydrukować kilka pierwszych dokumentów, aby sprawdzić
        # print("Pierwsze 5 dokumentów:", documents[:5])
    else:
        print(f"Plik '{csv_file_path}' został wczytany, ale wydaje się być pusty.")
        documents = [] # Ustaw pustą listę

except FileNotFoundError:
    print(f"Błąd: Plik '{csv_file_path}' nie został znaleziony.")
    print("Upewnij się, że plik znajduje się w odpowiednim miejscu lub podaj poprawną ścieżkę.")
    documents = [] # Ustaw pustą listę w razie błędu
except Exception as e:
    print(f"Wystąpił błąd podczas wczytywania pliku CSV: {e}")
    documents = [] # Ustaw pustą listę w razie błędu


# Teraz zmienna 'documents' zawiera listę zdań wczytanych z pliku CSV
# Jeśli wystąpił błąd wczytywania, 'documents' będzie pustą listą

Pomyślnie wczytano plik: sentences.csv
Wczytano 107 dokumentów.


In [17]:
# Przykład, stwórz własny zbiór!
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
document_embeddings = [embedding_model.embed_query(doc) for doc in documents]
document_embeddings_np = np.array(document_embeddings).astype('float32')

index = faiss.IndexFlatL2(document_embeddings_np.shape[1])
index.add(document_embeddings_np)

<ipython-input-17-4539c77e9445>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
def generate_answer(query, top_k=2):
    query_embedding = embedding_model.embed_query(query)
    query_embedding_np = np.array(query_embedding).reshape(1, -1).astype('float32')

    distances, indices = index.search(query_embedding_np, top_k)
    retrieved_docs = " ".join([documents[i] for i in indices[0]])

    input_text = f"question: {query} context: {retrieved_docs}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output = model.generate(input_ids)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

In [23]:
# Przykładowe testy (zmień pytania na swoje)

questions = [
     "In which city can you visit the Eiffel Tower?",
     "Where is the Colosseum located?",
     "Which city is home to the Sydney Opera House?",
     "What famous landmark can be found in New York City?",
     "Which European city features the Brandenburg Gate?"
]

for question in questions:
    answer = generate_answer(question)
    print(f"Q: {question}\nA: {answer}\n")

Q: In which city can you visit the Eiffel Tower?
A: Paris

Q: Where is the Colosseum located?
A: The Colosseum is a symbol of ancient Rome

Q: Which city is home to the Sydney Opera House?
A: Bondi Beach

Q: What famous landmark can be found in New York City?
A: 

Q: Which European city features the Brandenburg Gate?
A: Berlin



## Raport PDF
**Twoje zadanie:** Przygotuj krótki raport PDF (1-2 strony) z opisem:
- Jak przygotowałeś dane?
- Jak wyglądały wyniki?
- Jak działa RAG?  
- Jakie widzisz ograniczenia RAG?  
- Jakie są Twoje refleksje po eksperymencie z fine-tuningiem?   
- Jak zmieniły się odpowiedzi po fine-tuningu?
- Jak zmiana parametru k może wpłynąć na wyniki?